# Modeling Notebook
---

## Import Depedencies

In [10]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras.layers import LSTM, BatchNormalization, Bidirectional
from tensorflow.keras.layers import Layer

import numpy as np
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import plotly.express as px

import win_precise_time as wpt
import h5py

> Command to start Tensorboard: `tensorboard --logdir <log-location>`

## Modeling

In [2]:
NUM_LABELS = 9
BATCH_SIZE = 32
SAVE_DIR = 'models' 
LOG_DIR = 'logs'

In [3]:
with h5py.File('data/.preprocessed_data/datasets1.h5','r') as hf:
    y_train = np.array(hf.get('y_train'))
    y_val = np.array(hf.get('y_val'))

In [4]:
OHE = OneHotEncoder(sparse=False).fit(y_train.reshape(-1, 1))

decoder = {i : sign for i, sign in enumerate(OHE.categories_[0])}
decoder

c:\Users\Wildan Mufid R\miniconda3\envs\tf_gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


{0: b'batuk',
 1: b'demam',
 2: b'gigi',
 3: b'kepala',
 4: b'minum',
 5: b'obat',
 6: b'perut',
 7: b'resep',
 8: b'sakit'}

In [5]:
y_train = OHE.transform(y_train.reshape(-1, 1))
y_val = OHE.transform(y_val.reshape(-1, 1))

print(y_train.shape)
print(y_val.shape)

(816, 9)
(205, 9)


### ML Model

* LSTM Layer

In [6]:
class LSTM(Layer):
    def __init__(self, units, return_sequences=False, **kwargs):
        super(LSTM, self).__init__(**kwargs)
        self.units = units
        self.return_sequences = return_sequences

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.W_in = self.add_weight("W_in", (input_dim, self.units * 4), initializer="glorot_uniform", trainable=True)
        self.W_hiddenState = self.add_weight(
            "W_hiddenState", (self.units, self.units * 4), initializer="orthogonal", trainable=True
        )
        self.bias = self.add_weight("bias", (self.units * 4,), initializer="zeros", trainable=True)
        self.built = True

    def call(self, inputs, training=None):
        hidden = tf.zeros((inputs.shape[0], self.units))
        cell = tf.zeros((inputs.shape[0], self.units))
        output = []
        for i in range(inputs.shape[1]):
            x_in = inputs[:, i, :]
            z = tf.matmul(x_in, self.W_in) + tf.matmul(hidden, self.W_hiddenState) + self.bias
            zi, zf, zc, zo = tf.split(z, 4, axis=1)
            forget = tf.keras.activations.sigmoid(zf)
            inp = tf.keras.activations.sigmoid(zi)
            cell = forget * cell + inp * tf.keras.activations.tanh(zc)
            out = tf.keras.activations.sigmoid(zo)
            hidden = out * tf.keras.activations.tanh(cell)
            if self.return_sequences:
                output.append(hidden)
        if self.return_sequences:
            output = tf.stack(output, axis=1)
        else:
            output = hidden
        return output

* Bidirection LSTM Layer

In [7]:
class BidirectionalLSTM(Layer):
    def __init__(self, units, return_sequences=False, **kwargs):
        super(BidirectionalLSTM, self).__init__(**kwargs)
        self.units = units
        self.return_sequences = return_sequences

    def build(self, input_shape):       
        self.forward_lstm = LSTM(
            units=self.units,
            return_sequences=self.return_sequences,
        )
        
        self.backward_lstm = LSTM(
            units=self.units,
            return_sequences=self.return_sequences,
        )
        self.built = True

    def call(self, inputs):
        forward_output = self.forward_lstm(inputs)
        backward_output = self.backward_lstm(tf.reverse(inputs, axis=[1]))
        
        if self.return_sequences:
            backward_output = tf.reverse(backward_output, axis=[1])

        return tf.concat([forward_output, backward_output], axis=-1)

* Batch Normalization Layer

In [8]:
class BatchNormalization(Layer):
    def __init__(self, epsilon=1e-3, momentum=0.99, **kwargs):
        super(BatchNormalization, self).__init__(**kwargs)
        self.epsilon = epsilon
        self.momentum = momentum

    def build(self, input_shape):
        self.gamma = self.add_weight("gamma", shape=(input_shape[-1],), initializer="ones", trainable=True)
        self.beta = self.add_weight("beta", shape=(input_shape[-1],), initializer="zeros", trainable=True)
        self.moving_mean = self.add_weight("moving_mean", shape=(input_shape[-1],), initializer="zeros", trainable=False)
        self.moving_variance = self.add_weight("moving_variance", shape=(input_shape[-1],), initializer="ones", trainable=False)
        self.built = True

    def call(self, inputs, training=None):
        if training:
            batch_mean, batch_variance = tf.nn.moments(inputs, axes=[0], keepdims=False)
            
            update_moving_mean = tf.assign_sub(self.moving_mean, (1 - self.momentum) * (self.moving_mean - batch_mean))
            update_moving_variance = tf.assign_sub(self.moving_variance, (1 - self.momentum) * (self.moving_variance - batch_variance))
            
            with tf.control_dependencies([update_moving_mean, update_moving_variance]):
                normalized_inputs = (inputs - batch_mean) / tf.sqrt(batch_variance + self.epsilon)
        else:
            normalized_inputs = (inputs - self.moving_mean) / tf.sqrt(self.moving_variance + self.epsilon)
        
        scaled_inputs = self.gamma * normalized_inputs + self.beta
        return scaled_inputs

* Form the models

In [11]:
def lstm_model(input_shape):
    return tf.keras.models.Sequential([
        LSTM(512, return_sequences=True, input_shape=input_shape),
        LSTM(256, return_sequences=True),
        LSTM(128),
        BatchNormalization(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(NUM_LABELS, activation='softmax')
    ])
    

def bilstm_model(input_shape):
    return tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape),
        Bidirectional(LSTM(512, return_sequences=True, input_shape=input_shape)),
        Bidirectional(LSTM(256, return_sequences=True)),
        Bidirectional(LSTM(128)),
        BatchNormalization(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(NUM_LABELS, activation='softmax')
    ])

### Callbacks Function

In [10]:
class NoChangeStop(tf.keras.callbacks.Callback):
    def __init__(self, monitor='val_loss', patience=7, tolerance=.05):
        super(NoChangeStop, self).__init__()
        self.monitor = monitor
        self.patience = patience
        self.tolerance = tolerance
        self.wait = 0
        self.last_value = None

    def on_epoch_end(self, epoch, logs=None):
        current_value = logs.get(self.monitor)
        
        if self.last_value is None:
            self.last_value = current_value
        elif abs(current_value - self.last_value) <= self.tolerance:
            self.wait += 1
            if self.wait == self.patience:
                self.model.stop_training = True
        else:
            self.wait = 0
            self.last_value = current_value

In [11]:
noChangeStop = NoChangeStop(patience=7, tolerance=.05)

def checkpoint(fname):
    return ModelCheckpoint(f"{SAVE_DIR}/{fname}.h5", monitor='val_accuracy', mode='max', save_best_only=True)

### Metrics and Loss Function

* Categorical Crossentropy Loss

In [12]:
class CategoricalCrossentropy(tf.keras.losses.Loss):
    def __init__(self, name='custom_categorical_crossentropy', **kwargs):
        super(CategoricalCrossentropy, self).__init__(name=name, **kwargs)

    def call(self, y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
        
        loss = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1)
        
        return loss

* Accuracy Metric

In [13]:
class CategoricalAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='accuracy', **kwargs):
        super(CategoricalAccuracy, self).__init__(name=name, **kwargs)
        self.total = self.add_weight('total', initializer='zeros')
        self.count = self.add_weight('count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1)
        y_true = tf.argmax(y_true, axis=-1)
        values = tf.cast(tf.equal(y_true, y_pred), dtype=tf.float32)
       
        self.total.assign_add(tf.reduce_sum(values))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

* F1-Score Metric

In [14]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.true_positives = self.add_weight('true_positives', shape=(num_classes,), initializer='zeros')
        self.false_positives = self.add_weight('false_positives', shape=(num_classes,), initializer='zeros')
        self.false_negatives = self.add_weight('false_negatives', shape=(num_classes,), initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        for class_idx in range(self.num_classes):
            tp = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, class_idx), tf.equal(y_pred, class_idx)), dtype=tf.float32))
            fp = tf.reduce_sum(tf.cast(tf.logical_and(tf.not_equal(y_true, class_idx), tf.equal(y_pred, class_idx)), dtype=tf.float32))
            fn = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, class_idx), tf.not_equal(y_pred, class_idx)), dtype=tf.float32))
            
            self.true_positives[class_idx].assign_add(tp)
            self.false_positives[class_idx].assign_add(fp)
            self.false_negatives[class_idx].assign_add(fn)

    def result(self):
        epsilon = tf.keras.backend.epsilon()
        precision = self.true_positives / (self.true_positives + self.false_positives + epsilon)
        recall = self.true_positives / (self.true_positives + self.false_negatives + epsilon)
        f1 = 2.0 * (precision * recall) / (precision + recall + epsilon)
        return tf.reduce_mean(f1)

### Function Helper

In [15]:
def save_log(fname, history):
    with h5py.File(fname,'w') as hf:
        hf.create_dataset('accuracy', data = np.array(history.history['accuracy']))
        hf.create_dataset('val_accuracy', data = np.array(history.history['val_accuracy']))
        hf.create_dataset('f1_score', data = np.array(history.history['f1_score']))
        hf.create_dataset('val_f1_score', data = np.array(history.history['val_f1_score']))
        hf.create_dataset('loss', data = np.array(history.history['loss']))
        hf.create_dataset('val_loss', data = np.array(history.history['val_loss']))

In [16]:
def save_log(fname, history):
    with h5py.File(f'logs/{fname}.h5','w') as hf:
        hf.create_dataset('accuracy', data = np.array(history.history['accuracy']))
        hf.create_dataset('val_accuracy', data = np.array(history.history['val_accuracy']))
        hf.create_dataset('f1_score', data = np.array(history.history['f1_score']))
        hf.create_dataset('val_f1_score', data = np.array(history.history['val_f1_score']))
        hf.create_dataset('loss', data = np.array(history.history['loss']))
        hf.create_dataset('val_loss', data = np.array(history.history['val_loss']))

### Train Models

#### Datasets 1
Base Component (Face, Body, Right Hand, and Left Hand)

In [8]:
with h5py.File('data/.preprocessed_data/datasets1.h5','r') as hf:
    x_train = np.array(hf.get('x_train'))
    x_val = np.array(hf.get('x_val'))

##### LSTM

In [ ]:
model = lstm_model((x_train.shape[1],x_train.shape[2]))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 34, 512)           4454400   
                                                                 
 lstm_1 (LSTM)               (None, 34, 256)           787456    
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                        

In [23]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss= CategoricalCrossentropy(),
              metrics=[
                  CategoricalAccuracy(),
                  F1Score(NUM_LABELS),])

In [24]:
history = model.fit(x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    epochs = 500, 
                    batch_size=32,
                    callbacks=[noChangeStop, checkpoint('best_lstm_dset1')])

Epoch 1/500
26/26 [==============================] - 12s 95ms/step - loss: 1.6654 - accuracy: 0.4118 - f1_score: 0.3505 - val_loss: 1.6802 - val_accuracy: 0.5171 - val_f1_score: 0.4772 - relative: 20.9934
Epoch 2/500
26/26 [==============================] - 1s 52ms/step - loss: 1.0567 - accuracy: 0.6495 - f1_score: 0.6368 - val_loss: 1.4372 - val_accuracy: 0.5902 - val_f1_score: 0.6088 - relative: 22.0584
Epoch 3/500
26/26 [==============================] - 1s 54ms/step - loss: 0.8251 - accuracy: 0.7341 - f1_score: 0.7295 - val_loss: 1.1231 - val_accuracy: 0.7220 - val_f1_score: 0.7301 - relative: 23.4442
Epoch 4/500
26/26 [==============================] - 1s 36ms/step - loss: 0.7015 - accuracy: 0.7537 - f1_score: 0.7496 - val_loss: 0.9962 - val_accuracy: 0.6976 - val_f1_score: 0.7103 - relative: 24.8342
Epoch 5/500
26/26 [==============================] - 1s 37ms/step - loss: 0.5986 - accuracy: 0.7892 - f1_score: 0.7868 - val_loss: 0.9150 - val_accuracy: 0.6732 - val_f1_score: 0.6668

In [27]:
save_log('history_lstm_dset1', history)
model.save('models/lstm_dset1.h5')

In [28]:
tf.keras.backend.clear_session()
model.reset_states()
del model

##### Bi-LSTM

In [42]:
model = bilstm_model((x_train.shape[1],x_train.shape[2]))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 34, 1024)         8908800   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 34, 512)          2623488   
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              656384    
 nal)                                                            
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                1

In [43]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[
                  tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                  tfa.metrics.F1Score(NUM_LABELS),])

In [44]:
history = model.fit(x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    epochs = 500, 
                    batch_size=32,
                    callbacks=[noChangeStop, checkpoint('best_bilstm_dset1')])

Epoch 1/500
26/26 [==============================] - 14s 174ms/step - loss: 1.6211 - accuracy: 0.4338 - f1_score: 0.4138 - val_loss: 1.6277 - val_accuracy: 0.5268 - val_f1_score: 0.5053 - relative: 13.8776
Epoch 2/500
26/26 [==============================] - 2s 89ms/step - loss: 1.0340 - accuracy: 0.6446 - f1_score: 0.6278 - val_loss: 1.3097 - val_accuracy: 0.5854 - val_f1_score: 0.5749 - relative: 15.7443
Epoch 3/500
26/26 [==============================] - 2s 96ms/step - loss: 0.7572 - accuracy: 0.7341 - f1_score: 0.7232 - val_loss: 1.0835 - val_accuracy: 0.7073 - val_f1_score: 0.7078 - relative: 18.0683
Epoch 4/500
26/26 [==============================] - 2s 91ms/step - loss: 0.6043 - accuracy: 0.7978 - f1_score: 0.7916 - val_loss: 0.9213 - val_accuracy: 0.7122 - val_f1_score: 0.7121 - relative: 20.5283
Epoch 5/500
26/26 [==============================] - 2s 64ms/step - loss: 0.4792 - accuracy: 0.8444 - f1_score: 0.8406 - val_loss: 0.8121 - val_accuracy: 0.6878 - val_f1_score: 0.696

In [47]:
save_log('history_bilstm_dset1', history)
model.save('models/bilstm_dset1.h5')

In [48]:
tf.keras.backend.clear_session()
model.reset_states()
del model

#### Dataset 2
Component: Mouth, Body, Right Hand, and Left Hand

In [11]:
with h5py.File('data/.preprocessed_data/datasets2.h5','r') as hf:
    x_train = np.array(hf.get('x_train'))
    x_val = np.array(hf.get('x_val'))

##### LSTM Model

In [12]:
model = lstm_model((x_train.shape[1],x_train.shape[2]))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 34, 512)           1824768   
                                                                 
 lstm_1 (LSTM)               (None, 34, 256)           787456    
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                        

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[
                  tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                  tfa.metrics.F1Score(NUM_LABELS),])

In [14]:
history = model.fit(x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    epochs = 500, 
                    batch_size=32,
                    callbacks=[noChangeStop, checkpoint('best_lstm_dset2')])

Epoch 1/500
26/26 [==============================] - 11s 87ms/step - loss: 1.2811 - accuracy: 0.5551 - f1_score: 0.5407 - val_loss: 1.1597 - val_accuracy: 0.7220 - val_f1_score: 0.7215 - relative: 11.3592
Epoch 2/500
26/26 [==============================] - 1s 46ms/step - loss: 0.4578 - accuracy: 0.8664 - f1_score: 0.8647 - val_loss: 0.7387 - val_accuracy: 0.8390 - val_f1_score: 0.8475 - relative: 12.2456
Epoch 3/500
26/26 [==============================] - 1s 43ms/step - loss: 0.2317 - accuracy: 0.9289 - f1_score: 0.9284 - val_loss: 0.4006 - val_accuracy: 0.9366 - val_f1_score: 0.9395 - relative: 13.3869
Epoch 4/500
26/26 [==============================] - 1s 31ms/step - loss: 0.2053 - accuracy: 0.9375 - f1_score: 0.9374 - val_loss: 0.4049 - val_accuracy: 0.8878 - val_f1_score: 0.8897 - relative: 14.5442
Epoch 5/500
26/26 [==============================] - 1s 30ms/step - loss: 0.1750 - accuracy: 0.9473 - f1_score: 0.9469 - val_loss: 0.3435 - val_accuracy: 0.8976 - val_f1_score: 0.9042

In [17]:
save_log('history_lstm_dset2', history)
model.save('models/lstm_dset2.h5')

In [18]:
tf.keras.backend.clear_session()
model.reset_states()
del model

##### Bi-LSTM Model

In [19]:
model = bilstm_model((x_train.shape[1],x_train.shape[2]))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 34, 1024)         3649536   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 34, 512)          2623488   
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              656384    
 nal)                                                            
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                1

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[
                  tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                  tfa.metrics.F1Score(NUM_LABELS),])

In [21]:
history = model.fit(x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    epochs = 500, 
                    batch_size=32,
                    callbacks=[noChangeStop, checkpoint('best_bilstm_dset2')])

Epoch 1/500
26/26 [==============================] - 13s 152ms/step - loss: 1.0998 - accuracy: 0.6532 - f1_score: 0.6385 - val_loss: 0.8975 - val_accuracy: 0.8732 - val_f1_score: 0.8791 - relative: 13.0617
Epoch 2/500
26/26 [==============================] - 1s 51ms/step - loss: 0.2971 - accuracy: 0.8958 - f1_score: 0.8954 - val_loss: 0.5691 - val_accuracy: 0.8585 - val_f1_score: 0.8633 - relative: 14.5499
Epoch 3/500
26/26 [==============================] - 2s 89ms/step - loss: 0.2077 - accuracy: 0.9338 - f1_score: 0.9332 - val_loss: 0.4401 - val_accuracy: 0.9024 - val_f1_score: 0.9046 - relative: 15.9264
Epoch 4/500
26/26 [==============================] - 1s 53ms/step - loss: 0.1345 - accuracy: 0.9498 - f1_score: 0.9496 - val_loss: 0.3498 - val_accuracy: 0.8878 - val_f1_score: 0.8901 - relative: 18.2107
Epoch 5/500
26/26 [==============================] - 2s 80ms/step - loss: 0.1255 - accuracy: 0.9559 - f1_score: 0.9558 - val_loss: 0.3016 - val_accuracy: 0.9073 - val_f1_score: 0.911

In [25]:
save_log('history_bilstm_dset2', history)
model.save('models/bilstm_dset2.h5')

In [26]:
tf.keras.backend.clear_session()
model.reset_states()
del model

#### Dataset 3
Component: Face, Upper Body, Right Hand, and Left Hand

In [27]:
with h5py.File('data/.preprocessed_data/datasets3.h5','r') as hf:
    x_train = np.array(hf.get('x_train'))
    x_val = np.array(hf.get('x_val'))

##### LSTM

In [28]:
model = lstm_model((x_train.shape[1],x_train.shape[2]))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 34, 512)           4388864   
                                                                 
 lstm_1 (LSTM)               (None, 34, 256)           787456    
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                        

In [29]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[
                  tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                  tfa.metrics.F1Score(NUM_LABELS),])

In [30]:
history = model.fit(x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    epochs = 500, 
                    batch_size=32,
                    callbacks=[noChangeStop, checkpoint('best_lstm_dset3')])

Epoch 1/500
26/26 [==============================] - 7s 93ms/step - loss: 1.5482 - accuracy: 0.4645 - f1_score: 0.4202 - val_loss: 1.6450 - val_accuracy: 0.5659 - val_f1_score: 0.5491 - relative: 7.6512
Epoch 2/500
26/26 [==============================] - 1s 55ms/step - loss: 0.9786 - accuracy: 0.6532 - f1_score: 0.6354 - val_loss: 1.3825 - val_accuracy: 0.5951 - val_f1_score: 0.5919 - relative: 8.7283
Epoch 3/500
26/26 [==============================] - 1s 55ms/step - loss: 0.7585 - accuracy: 0.7390 - f1_score: 0.7337 - val_loss: 1.0604 - val_accuracy: 0.6878 - val_f1_score: 0.6966 - relative: 10.1542
Epoch 4/500
26/26 [==============================] - 1s 37ms/step - loss: 0.6560 - accuracy: 0.7635 - f1_score: 0.7574 - val_loss: 0.9529 - val_accuracy: 0.6634 - val_f1_score: 0.6698 - relative: 11.5833
Epoch 5/500
26/26 [==============================] - 1s 54ms/step - loss: 0.5629 - accuracy: 0.8002 - f1_score: 0.7978 - val_loss: 0.8508 - val_accuracy: 0.7073 - val_f1_score: 0.7114 - 

In [33]:
save_log('history_lstm_dset3', history)
model.save('models/lstm_dset3.h5')

In [37]:
tf.keras.backend.clear_session()
model.reset_states()
del model

##### Bi-LSTM

In [39]:
model = bilstm_model((x_train.shape[1],x_train.shape[2]))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 34, 1024)         8777728   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 34, 512)          2623488   
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              656384    
 nal)                                                            
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                1

In [40]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[
                  tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                  tfa.metrics.F1Score(NUM_LABELS),])

In [41]:
history = model.fit(x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    epochs = 500, 
                    batch_size=32,
                    callbacks=[noChangeStop, checkpoint('best_bilstm_dset3')])

Epoch 1/500
26/26 [==============================] - 14s 170ms/step - loss: 1.7387 - accuracy: 0.4277 - f1_score: 0.3694 - val_loss: 1.6666 - val_accuracy: 0.4732 - val_f1_score: 0.4511 - relative: 14.1277
Epoch 2/500
26/26 [==============================] - 3s 98ms/step - loss: 1.1785 - accuracy: 0.5956 - f1_score: 0.5706 - val_loss: 1.4141 - val_accuracy: 0.5171 - val_f1_score: 0.5307 - relative: 15.9859
Epoch 3/500
26/26 [==============================] - 2s 95ms/step - loss: 0.8922 - accuracy: 0.7034 - f1_score: 0.6935 - val_loss: 1.2468 - val_accuracy: 0.6341 - val_f1_score: 0.6457 - relative: 18.4913
Epoch 4/500
26/26 [==============================] - 3s 99ms/step - loss: 0.6890 - accuracy: 0.7586 - f1_score: 0.7547 - val_loss: 1.0597 - val_accuracy: 0.6780 - val_f1_score: 0.6876 - relative: 20.9204
Epoch 5/500
26/26 [==============================] - 2s 94ms/step - loss: 0.5576 - accuracy: 0.8051 - f1_score: 0.8021 - val_loss: 0.8823 - val_accuracy: 0.7317 - val_f1_score: 0.733

In [44]:
save_log('history_bilstm_dset3', history)
model.save('models/bilstm_dset3.h5')

In [45]:
tf.keras.backend.clear_session()
model.reset_states()
del model

#### Dataset 4
Component: Mouth, Upper Body, Right Hand, and Left Hand

In [11]:
with h5py.File('data/.preprocessed_data/datasets4.h5','r') as hf:
    x_train = np.array(hf.get('x_train'))
    x_val = np.array(hf.get('x_val'))

##### LSTM

In [26]:
model = lstm_model((x_train.shape[1],x_train.shape[2]))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 34, 512)           1759232   
                                                                 
 lstm_1 (LSTM)               (None, 34, 256)           787456    
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                        

In [27]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[
                  tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                  tfa.metrics.F1Score(NUM_LABELS),])

In [28]:
history = model.fit(x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    epochs = 500, 
                    batch_size=32,
                    callbacks=[noChangeStop, checkpoint('best_lstm_dset4')])

Epoch 1/500
26/26 [==============================] - 7s 83ms/step - loss: 0.9327 - accuracy: 0.6887 - f1_score: 0.6743 - val_loss: 0.9509 - val_accuracy: 0.8537 - val_f1_score: 0.8555 - relative: 7.2643
Epoch 2/500
26/26 [==============================] - 1s 42ms/step - loss: 0.3170 - accuracy: 0.8946 - f1_score: 0.8937 - val_loss: 0.5939 - val_accuracy: 0.8829 - val_f1_score: 0.8880 - relative: 8.1207
Epoch 3/500
26/26 [==============================] - 1s 31ms/step - loss: 0.2099 - accuracy: 0.9301 - f1_score: 0.9298 - val_loss: 0.4953 - val_accuracy: 0.8732 - val_f1_score: 0.8763 - relative: 9.2487
Epoch 4/500
26/26 [==============================] - 2s 59ms/step - loss: 0.2016 - accuracy: 0.9277 - f1_score: 0.9271 - val_loss: 0.3278 - val_accuracy: 0.9171 - val_f1_score: 0.9217 - relative: 10.0514
Epoch 5/500
26/26 [==============================] - 1s 31ms/step - loss: 0.1517 - accuracy: 0.9498 - f1_score: 0.9490 - val_loss: 0.3882 - val_accuracy: 0.8927 - val_f1_score: 0.8975 - r

In [31]:
save_log('history_lstm_dset4', history)
model.save('models/lstm_dset4.h5')

In [32]:
tf.keras.backend.clear_session()
model.reset_states()
del model

##### Bi-LSTM

In [12]:
model = bilstm_model((x_train.shape[1],x_train.shape[2]))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 34, 1024)         3518464   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 34, 512)          2623488   
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              656384    
 nal)                                                            
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                1

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[
                  tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                  tfa.metrics.F1Score(NUM_LABELS),])

In [14]:
history = model.fit(x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    epochs = 500, 
                    batch_size=32,
                    callbacks=[noChangeStop, checkpoint('best_bilstm_dset4')])

Epoch 1/500
26/26 [==============================] - 15s 150ms/step - loss: 1.0183 - accuracy: 0.6863 - f1_score: 0.6773 - val_loss: 0.9091 - val_accuracy: 0.8146 - val_f1_score: 0.8220 - relative: 16.9823
Epoch 2/500
26/26 [==============================] - 2s 76ms/step - loss: 0.3371 - accuracy: 0.8934 - f1_score: 0.8929 - val_loss: 0.4755 - val_accuracy: 0.9220 - val_f1_score: 0.9239 - relative: 18.4513
Epoch 3/500
26/26 [==============================] - 2s 83ms/step - loss: 0.1724 - accuracy: 0.9522 - f1_score: 0.9521 - val_loss: 0.3389 - val_accuracy: 0.9317 - val_f1_score: 0.9370 - relative: 20.4627
Epoch 4/500
26/26 [==============================] - 1s 54ms/step - loss: 0.1638 - accuracy: 0.9547 - f1_score: 0.9540 - val_loss: 0.2563 - val_accuracy: 0.9317 - val_f1_score: 0.9343 - relative: 22.6731
Epoch 5/500
26/26 [==============================] - 2s 83ms/step - loss: 0.1142 - accuracy: 0.9718 - f1_score: 0.9715 - val_loss: 0.2533 - val_accuracy: 0.9366 - val_f1_score: 0.940

In [17]:
save_log('history_bilstm_dset4', history)
model.save('models/bilstm_dset4.h5')

In [18]:
tf.keras.backend.clear_session()
model.reset_states()
del model

#### Dataset 5
Component: Right Hand and Left Hand

In [33]:
with h5py.File('data/.preprocessed_data/datasets5.h5','r') as hf:
    x_train = np.array(hf.get('x_train'))
    x_val = np.array(hf.get('x_val'))

##### LSTM

In [34]:
model = lstm_model((x_train.shape[1],x_train.shape[2]))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 34, 512)           1308672   
                                                                 
 lstm_1 (LSTM)               (None, 34, 256)           787456    
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                        

In [35]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[
                  tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                  tfa.metrics.F1Score(NUM_LABELS),])

In [36]:
history = model.fit(x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    epochs = 500, 
                    batch_size=32,
                    callbacks=[noChangeStop, checkpoint('best_lstm_dset5')])

Epoch 1/500
26/26 [==============================] - 7s 83ms/step - loss: 0.8817 - accuracy: 0.7292 - f1_score: 0.7284 - val_loss: 0.9407 - val_accuracy: 0.8732 - val_f1_score: 0.8731 - relative: 6.8311
Epoch 2/500
26/26 [==============================] - 1s 29ms/step - loss: 0.3723 - accuracy: 0.8738 - f1_score: 0.8724 - val_loss: 0.6888 - val_accuracy: 0.8488 - val_f1_score: 0.8480 - relative: 7.6494
Epoch 3/500
26/26 [==============================] - 1s 30ms/step - loss: 0.2513 - accuracy: 0.9240 - f1_score: 0.9237 - val_loss: 0.5000 - val_accuracy: 0.8732 - val_f1_score: 0.8802 - relative: 8.4258
Epoch 4/500
26/26 [==============================] - 1s 43ms/step - loss: 0.2422 - accuracy: 0.9326 - f1_score: 0.9322 - val_loss: 0.3899 - val_accuracy: 0.9220 - val_f1_score: 0.9235 - relative: 9.1832
Epoch 5/500
26/26 [==============================] - 1s 29ms/step - loss: 0.1618 - accuracy: 0.9485 - f1_score: 0.9481 - val_loss: 0.3571 - val_accuracy: 0.9073 - val_f1_score: 0.9126 - re

In [39]:
save_log('history_lstm_dset5', history)
model.save('models/lstm_dset5.h5')

In [40]:
tf.keras.backend.clear_session()
model.reset_states()
del model

##### Bi-LSTM

In [41]:
model = bilstm_model((x_train.shape[1],x_train.shape[2]))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 34, 1024)         2617344   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 34, 512)          2623488   
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              656384    
 nal)                                                            
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                1

In [42]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[
                  tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                  tfa.metrics.F1Score(NUM_LABELS),])

In [43]:
history = model.fit(x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    epochs = 500, 
                    batch_size=32,
                    callbacks=[noChangeStop, checkpoint('best_bilstm_dset5')])

Epoch 1/500
26/26 [==============================] - 13s 152ms/step - loss: 0.9147 - accuracy: 0.6985 - f1_score: 0.6980 - val_loss: 0.9232 - val_accuracy: 0.8146 - val_f1_score: 0.8171 - relative: 12.8369
Epoch 2/500
26/26 [==============================] - 2s 81ms/step - loss: 0.3850 - accuracy: 0.8836 - f1_score: 0.8832 - val_loss: 0.5014 - val_accuracy: 0.9220 - val_f1_score: 0.9237 - relative: 14.3072
Epoch 3/500
26/26 [==============================] - 1s 50ms/step - loss: 0.2240 - accuracy: 0.9338 - f1_score: 0.9335 - val_loss: 0.3988 - val_accuracy: 0.9220 - val_f1_score: 0.9254 - relative: 16.3668
Epoch 4/500
26/26 [==============================] - 1s 50ms/step - loss: 0.2368 - accuracy: 0.9130 - f1_score: 0.9126 - val_loss: 0.4640 - val_accuracy: 0.8390 - val_f1_score: 0.8459 - relative: 17.6612
Epoch 5/500
26/26 [==============================] - 2s 80ms/step - loss: 0.1807 - accuracy: 0.9400 - f1_score: 0.9393 - val_loss: 0.2167 - val_accuracy: 0.9463 - val_f1_score: 0.948

In [46]:
save_log('history_bilstm_dset5', history)
model.save('models/bilstm_dset5.h5')

In [47]:
tf.keras.backend.clear_session()
model.reset_states()
del model

### Purge

In [ ]:
for red in np.arange(1,6):
    with h5py.File(f'data/.preprocessed_data/datasets{red}.h5','r') as hf:
            x_train = np.array(hf.get('x_train'))
            x_val = np.array(hf.get('x_val'))
            
    for bz in [16, 32, 64]:
        for lr in [1E-3,1E-4,1E-5]:
            for mod in ['lstm','bilstm']:
                if 'lstm':
                    model = lstm_model((x_train.shape[1],x_train.shape[2]))
                elif 'bilstm':
                    model = bilstm_model((x_train.shape[1],x_train.shape[2])) 
        
                model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                        loss=tf.keras.losses.CategoricalCrossentropy(),
                        metrics=[
                            tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                            tfa.metrics.F1Score(NUM_LABELS),])
        
                history = model.fit(x_train,
                                y_train,
                                validation_data=(x_val, y_val),
                                epochs = 500, 
                                batch_size=bz,
                                callbacks=[noChangeStop, 
                                        checkpoint(f"models3/v2/{bz}/best_{mod}_dset{red}_{lr}.h5")])
                
                save_log(f'logs3/v2/{bz}/history_{mod}_dset{red}_{lr}.h5', history)
                model.save(f"models3/v2/{bz}/{mod}_dset{red}_{lr}.h5")
                
                tf.keras.backend.clear_session()
                model.reset_states()
                del model